# Тестовое задание в команду Data Science IDP

Импорт библиотек

In [85]:
import pandas as pd

import os
import pathlib
import re

from typing import List, Tuple

## Формулировка задания

**Написать решение по извлечению сущностей из документов (новостных текстов)**

**Датасет** -- семпл текстов с аннотациями русских новостей (источник: https://bsnlp.cs.helsinki.fi/bsnlp-2019/shared_task.html)

**Интересующие сущности:**
- *PER* - persons (личности)
- *ORG* - organizations (организации)
- *LOC* - locations (места)
- *EVT* - events (события)
- *PRO* - products (продукты и документы)

Достаточно использовать *9 документов* про *брекзит* из семпла

## Задание 1 (Анализ задачи)

**Описание задачи с точки зрения NLP**

Это задача извлечения именованных сущностей из текст -- *Named Entity Recognition (NER)*. В данном случае мы занимаемся последовательной разметкой, классифицируя каждый токен как PER, ORG, LOC, EVT, PRO или не являющийся сущностью

**Классические методы решения:**
- Статистические методы: *Условные случайные поля (CRF)*, *Марковская модель максимальной энтропии (MEMM)*
- Рекуррентные нейронные сети: *LSTM*, *BiLSTM* (часто с CRF слоем)
- Энкодерные трансформеры: *BERT-like*

**LLM методы:**
- *Zero-shot/Few-shot* подходы (GPT, в т.ч. *GigaChat*)
- Промты с инструкциями для GPT-моделей
- *Fine-tuning* на обучающей выборке
- *Chain-of-Thought (CoT)* промпты для размышления перед определением сущности

**Метрики качества:**
- *Precision, Recall и F1-score* для найденных сущностей (верные границы), **без учета типа сущности**
- *Precision, Recall и F1-score* для **точного совпадения** (границы + тип сущности)
- Эти же метрики с *частичным совпадением границ*
- Эти же метрики для каждого отдельного класса (для выявления, с какими типами сущностей у модели проблемы). Macro-F1 (усреднённый по классам)

- Можно добавить *accuracy* лемматизации каждой верно найденной сущности (для задачи из оригинального соревнования)
- *Inference time* (в данном случае зависит от провайдера GigaChat)

## Задание 2 (Формирование DataFrame)

### Чтение датасета в pandas DataFrame с обязательными колонками "document_id", "document_text", "entity", "gold_answer"

#### Функция для извлечения данных и формирования DataFrame

In [86]:
def read_bsnlp_data(data_dir: str='sample_pl_cs_ru_bg', lang: str="ru") -> pd.DataFrame:
    """
    Считывание данных соревнования из папок raw/ru и annotated/ru и преобразует в
    требуемый формат pd.DataFrame: ["document_id", "document_text", "entity", "gold_answer"]
    """
    raw_dir = pathlib.Path(data_dir) / 'raw' / lang
    annotated_dir = pathlib.Path(data_dir) / 'annotated' / lang

    documents_desc: List[dict] = []

    # Raw файлы
    for raw_file in raw_dir.glob('*.txt'):
        with open(raw_file, encoding="utf-8") as f:
            lines: List[str] = f.readlines()

        assert len(lines) >= 5, f"File \"{str(raw_file)}\" contains less than 5 lines"

        document_id = lines[0].strip()
        language = lines[1].strip()
        date = lines[2].strip()
        url = lines[3].strip()
        title = lines[4].strip()
        content = ' '.join([line.strip() for line in lines[5:] if line.strip()])
        document_text = title + ' ' + content

        # Ищем соответствующий аннотированный файл
        annotation_file = annotated_dir / f"{raw_file.stem}.out"

        assert annotation_file.exists(), \
            f"Annotation file \"{str(annotation_file)}\" corresponding to raw file \"{str(raw_file)}\" does not exist"
        
        # Читаем аннотации
        with open(annotation_file, encoding='utf-8') as f:
            annotation_lines = f.readlines()

        assert len(annotation_lines) > 0, f"Annotation file \"{str(annotation_file)}\" is empty"
        annotation_doc_id = annotation_lines[0].strip()

        assert document_id == annotation_doc_id, "Raw and Annotated doc ids do not match: " \
                                                f"{document_id}, {annotation_doc_id}"

        entities_found = []

        for line in annotation_lines[1:]:
            line = line.strip()
            if line:
                columns = line.split('\t')
                assert len(columns) == 4

                entity_text, entity_text_lemmatized, entity_type, entity_id = columns

                entities_found.append( (entity_text, entity_type) )
        
        for entity_text, entity_type in entities_found:
            documents_desc.append({
                'document_id': document_id,
                'document_text': document_text,
                'entity': entity_text,
                'gold_answer': entity_type
            })
        if not entities_found:
            documents_desc.append({
                'document_id': document_id,
                'document_text': document_text,
                'entity': None,
                'gold_answer': None
            })
        
    return pd.DataFrame(documents_desc)


#### Просмотр данных

In [87]:
df = read_bsnlp_data()
df.head()

,document_id,document_text,entity,gold_answer
0,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Brexit,EVT
1,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Альбиона,LOC
2,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Альбионе,LOC
3,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Борис Джонсон,PER
4,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Британии,LOC


## Задание 3 (Функция для преобразования строки из DataFrame в запрос для LLM)

In [88]:
def create_prompt_for_llm(row: pd.Series) -> str:
    """
    Создает промпт для LLM на основе строки датафрейма
    """
    document_text = row['document_text']
    
    prompt = f"""Извлеки из **текста для анализа** все (необязательно уникальные) именованные сущности и классифицируй их по типам, опираясь на контекст и нижеуказанные инструкции.

**Типы сущностей:**
- PER (persons) - личности, имена людей (без прилежащих титулов), группы людей ("русские"; НО НЕ "мусульмане", т.к. нет конкретной объединяющей организации)
- ORG (organizations) - организации, компании, учреждения, объединения
- LOC (locations) - географические места, страны, города
- EVT (events) - события, процессы, явления
- PRO (products) - продукты, документы, произведения

**Если равновероятными кажутся типы:**
- ORG и PER, то должно быть выбрано PER;
- ORG и PRO, то должно быть выбрано ORG.

**Замечания:**
- Сохраняй точную форму, регистр слов и количество вхождений сущностей как в оригинальном тексте. 
То есть, если сущность встречается в нескольких формах (Москва, Москвы), каждая форма должна быть в ответе.
Однако одинаковые формы (Москва, Москва) повторять не надо.
- Текст "Премьер-министр Великобритании Тереза Мэй" содержит LOC (Великобритании) и PER (Тереза Мэй)
- Текст "Церковь Св. Стефана в Стамбуле" является целиком одной сущностью типа LOC
- Расположение организации ORG и события EVT является частью их сущности ("Сберанк в Москве" - ORG, "Олимпиада 2014 в Сочи" - EVT)
- Если событие так же является местом ("Он погиб в Сталинграде, незадолго до битвы"), то выбирается тип LOC

**Текст для анализа:**
{document_text}

**Формат ответа:**
Перечисли найденные сущности в формате:
- [Название сущности] -- [Тип];

Пример:
- Борис Джонсон -- PER;
- Британия -- LOC;
- Британии -- LOC;
- Brexit -- EVT"""

    return prompt

In [89]:
# Тестируем функцию на первой строке
test_row = df.iloc[0]
test_prompt = create_prompt_for_llm(test_row)
print("Пример промпта для первой строки:")
print("="*50)
print(test_prompt)
print("="*50)


Пример промпта для первой строки:
Извлеки из **текста для анализа** все (необязательно уникальные) именованные сущности и классифицируй их по типам, опираясь на контекст и нижеуказанные инструкции.

**Типы сущностей:**
- PER (persons) - личности, имена людей (без прилежащих титулов), группы людей ("русские"; НО НЕ "мусульмане", т.к. нет конкретной объединяющей организации)
- ORG (organizations) - организации, компании, учреждения, объединения
- LOC (locations) - географические места, страны, города
- EVT (events) - события, процессы, явления
- PRO (products) - продукты, документы, произведения

**Если равновероятными кажутся типы:**
- ORG и PER, то должно быть выбрано PER;
- ORG и PRO, то должно быть выбрано ORG.

**Замечания:**
- Сохраняй точную форму, регистр слов и количество вхождений сущностей как в оригинальном тексте. 
То есть, если сущность встречается в нескольких формах (Москва, Москвы), каждая форма должна быть в ответе.
Однако одинаковые формы (Москва, Москва) повторять не 

## Задание 4 (Получение и сохранение ответов от GigaChat)

### Функции для реализации интерфейса в jupyter-notebook

In [90]:
def split_text_into_chunks(text: str, max_words: int = 300) -> List[str]:
    """
    Разбивает текст на части по предложениям
    """
    sentences = text.split('.')
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
            
        # Проверяем, не превысит ли добавление предложения лимит
        test_chunk = current_chunk + ". " + sentence if current_chunk else sentence
        if len(test_chunk.split()) <= max_words:
            current_chunk = test_chunk
        else:
            if current_chunk:
                chunks.append(current_chunk + ".")
            current_chunk = sentence
    
    if current_chunk:
        chunks.append(current_chunk + ".")
    
    return chunks


def get_llm_response_interface(df: pd.DataFrame) -> pd.DataFrame:
    """
    Интерфейс для получения ответов от LLM и сохранения результатов
    """
    # Создаем копию DataFrame и добавляем новые колонки
    result_df = df.copy()
    result_df['llm_entity'] = None
    result_df['llm_answer'] = None
    
    # Получаем уникальные документы для обработки
    unique_docs = df['document_id'].unique()
    
    print(f"Найдено {len(unique_docs)} уникальных документов для обработки:")
    print(list(unique_docs))
    
    return result_df

def parse_llm_response(response: str) -> List[Tuple[str, str]]:
    """
    Парсит ответ LLM и возвращает список (entity, type)
    """
    entities = []
    lines = response.strip().split(';')
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        # Убираем тире в начале
        if line.startswith('-'):
            line = line[1:].strip()
            
        # Ищем паттерн "сущность -- тип"
        if ' -- ' in line:
            parts = line.split(' -- ', 1)
            if len(parts) == 2:
                entity = parts[0].strip()
                entity_type = parts[1].strip()
                if entity and entity_type:
                    entities.append((entity, entity_type))
    
    return entities

def process_document_chunked(doc_id: str, df: pd.DataFrame) -> pd.DataFrame:
    """
    Обрабатывает один документ по частям: показывает промпт и позволяет ввести ответ LLM
    """
    # Получаем строки для данного документа
    doc_rows = df[df['document_id'] == doc_id]
    if doc_rows.empty:
        raise Exception(f"Документ {doc_id} не найден!")
    
    first_row = doc_rows.iloc[0]

    document_text = first_row['document_text']
    
    # Разбиваем текст на части
    chunks = split_text_into_chunks(document_text, max_words=250)
    
    print(f"Документ {doc_id} разбит на {len(chunks)} частей")
    
    all_llm_entities = []
    
    for i, chunk in enumerate(chunks, 1):
        print(f"\n--- ЧАСТЬ {i}/{len(chunks)} ---")
        
        # Создаем временную строку для части текста
        chunk_row = first_row.copy()
        chunk_row['document_text'] = chunk
        
        prompt = create_prompt_for_llm(chunk_row)
        
        print("ПРОМПТ:")
        print("-" * 40)
        print(prompt)
        print("-" * 40)
        
        print(f"\nВведите ответ для части {i} (пустая строка для завершения):")
        
        llm_response_lines = []
        while True:
            line = input()
            if line.strip() == '':
                break
            llm_response_lines.append(line)
        
        if llm_response_lines:
            llm_response = '\n'.join(llm_response_lines)
            chunk_entities = parse_llm_response(llm_response)
            all_llm_entities.extend(chunk_entities)
    
    # Удаляем дубликаты
    unique_entities = []
    seen = set()
    for entity, entity_type in all_llm_entities:
        entity_key = (entity.lower(), entity_type)
        if entity_key not in seen:
            unique_entities.append((entity, entity_type))
            seen.add(entity_key)
    
    print(f"\nОбъединенный результат: {len(unique_entities)} уникальных сущностей")
    
    # Сопоставляем найденные LLM сущности с золотым стандартом
    gold_entities_list = doc_rows[doc_rows['entity'].notna()]['entity'].tolist()
    matched_entities = set()
    new_rows = []
    
    for llm_entity, llm_type in unique_entities:
        # Ищем точное совпадение с золотым стандартом
        match_found = False
        for i, gold_entity in enumerate(gold_entities_list):
            if llm_entity == gold_entity and gold_entity not in matched_entities:
                # Найдено совпадение - обновляем соответствующую строку
                mask = (df['document_id'] == doc_id) & (df['entity'] == gold_entity)
                df.loc[mask, 'llm_entity'] = llm_entity
                df.loc[mask, 'llm_answer'] = llm_type
                matched_entities.add(gold_entity)
                match_found = True
                break
        
        if not match_found:
            # LLM нашла новую сущность - создаем новую строку
            new_row = {
                'document_id': doc_id,
                'document_text': first_row['document_text'],
                'entity': None,
                'gold_answer': None,
                'llm_entity': llm_entity,
                'llm_answer': llm_type
            }
            new_rows.append(new_row)
    
    
    print(f"\nРезультат сохранен для документа {doc_id}")
    print(f"LLM нашла {len(unique_entities)} сущностей")
    print(f"Совпадений с золотым стандартом: {len(matched_entities)}")
    print(f"Новых сущностей от LLM: {len(new_rows)}")
    print("="*80)

    # Добавляем новые строки для сущностей, найденных только LLM
    if new_rows:
        new_df = pd.DataFrame(new_rows)
        # Используем pd.concat для добавления новых строк
        df = pd.concat([df, new_df], ignore_index=True)
    
    return df

def process_all_documents(df: pd.DataFrame) -> pd.DataFrame:
    """
    Последовательно обрабатывает все документы
    """
    unique_docs = df['document_id'].unique()
    
    print(f"Начинаем обработку {len(unique_docs)} документов...")
    print("Для каждого документа будет показан промпт, который нужно отправить в LLM")
    print("Затем введите полученный ответ")
    print("\n")
    
    for i, doc_id in enumerate(unique_docs, 1):
        print(f"\n[{i}/{len(unique_docs)}]")
        df = process_document_chunked(doc_id, df)
        
        if i < len(unique_docs):
            ans = input(f"\nПродолжить с документом {i+1}? (y/n): ").strip().lower()
            if ans.lower() != 'y':
                print("Обработка остановлена.")
                break
    
    return df


### Печать пропмтов и получение ответов GigaChat

In [91]:
# Инициализируем DataFrame для работы
result_df = get_llm_response_interface(df)

Найдено 9 уникальных документов для обработки:
['ru-10', 'ru-1000', 'ru-1001', 'ru-1002', 'ru-1003', 'ru-1004', 'ru-1006', 'ru-1011', 'ru-1017']


In [92]:
result_df = process_all_documents(result_df)

Начинаем обработку 9 документов...
Для каждого документа будет показан промпт, который нужно отправить в LLM
Затем введите полученный ответ



[1/9]
Документ ru-10 разбит на 3 частей

--- ЧАСТЬ 1/3 ---
ПРОМПТ:
----------------------------------------
Извлеки из **текста для анализа** все (необязательно уникальные) именованные сущности и классифицируй их по типам, опираясь на контекст и нижеуказанные инструкции.

**Типы сущностей:**
- PER (persons) - личности, имена людей (без прилежащих титулов), группы людей ("русские"; НО НЕ "мусульмане", т.к. нет конкретной объединяющей организации)
- ORG (organizations) - организации, компании, учреждения, объединения
- LOC (locations) - географические места, страны, города
- EVT (events) - события, процессы, явления
- PRO (products) - продукты, документы, произведения

**Если равновероятными кажутся типы:**
- ORG и PER, то должно быть выбрано PER;
- ORG и PRO, то должно быть выбрано ORG.

**Замечания:**
- Сохраняй точную форму, регистр слов и коли

### Сохранение Датафрейма

In [94]:
def save_results(df: pd.DataFrame, filename: str = 'llm_results_df'):
    """Сохраняет результаты в нескольких форматах"""
    df.to_pickle(f'{filename}.pkl')
    df.to_csv(f'{filename}.csv', index=False, encoding='utf-8')
    print(f"Результаты сохранены как {filename}.pkl и {filename}.csv")


def load_results(filename: str = 'llm_results') -> pd.DataFrame:
    """Загружает сохраненные результаты"""
    try:
        df = pd.read_pickle(f'{filename}.pkl')
        print(f"Загружен {filename}.pkl")
        return df
    except FileNotFoundError:
        try:
            df = pd.read_csv(f'{filename}.csv', encoding='utf-8')
            print(f"Загружен {filename}.csv")
            return df
        except FileNotFoundError:
            raise FileNotFoundError(f"Файлы {filename}.pkl или {filename}.csv не найдены")

In [95]:
save_results(result_df)

Результаты сохранены как llm_results_df.pkl и llm_results_df.csv


### Просмотр результатов

In [96]:
result_df.head(10)

,document_id,document_text,entity,gold_answer,llm_entity,llm_answer
0,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Brexit,EVT,Brexit,EVT
1,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Альбиона,LOC,None,None
2,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Альбионе,LOC,None,None
3,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Борис Джонсон,PER,Борис Джонсон,PER
4,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Британии,LOC,Британии,LOC
5,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Британия,LOC,Британия,LOC
6,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Брюсселе,LOC,None,None
7,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Брюсселем,LOC,None,None
8,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Брюссель,LOC,Брюссель,LOC
9,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Брюсселю,LOC,None,None


## Задание 5 (Подсчёт метрик)

### Класс для подсчёта некоторых метрик

In [ ]:
import numpy as np
import pandas as pd


class NERMetrics:
    """
    Класс для вычисления метрик качества распознавания сущностей
    """
    @staticmethod
    def extract_entity(label: str) -> str | None:
        """
        Извлечение сущности из строки метки

        Args:
            label (str | None): метка, строка вида "entity|type"
        Returns:
            entity (str | None): строка сущности
        """
        if pd.isna(label):
            return None
        return label.split('|')[0] if '|' in label else None
    
    @staticmethod
    def extract_type(label: str) -> str | None:
        """
        Извлечение типа сущности из строки метки

        Args:
            label (str | None): метка, строка вида "entity|type"
        Returns:
            type (str | None): строка типа сущности
        """
        if pd.isna(label):
            return None
        return label.split('|')[1] if '|' in label else None

    @staticmethod
    def exact_match_score(gold: str, pred: str) -> float:
        """
        Точное совпадение сущности и типа

        Args:
            gold (str | None): истинное значение, строка вида "entity|type"
            pred (str | None): предсказанное значение, строка вида "entity|type"
        Returns:
            score (float): 1.0 если точное совпадение, 0.0 иначе
        """
        if pd.isna(gold) and pd.isna(pred):
            return 1.
        if pd.isna(gold) or pd.isna(pred):
            return 0.
        return 1. if gold == pred else 0.
    
    @staticmethod
    def boundary_match_score(gold: str, pred: str) -> float:
        """
        Совпадение границ и формы сущности без учёта типа
        
        Args:
            gold (str | None): истинное значение, строка вида "entity|type"
            pred (str | None): предсказанное значение, строка вида "entity|type"
        Returns:
            score (float): 1.0 если строки одинаковы, 0.0 иначе
        """
        if pd.isna(gold) and pd.isna(pred):
            return 1.
        if pd.isna(gold) or pd.isna(pred):
            return 0.
        
        # Извлечение сущностей
        gold_entity = NERMetrics.extract_entity(gold)
        pred_entity = NERMetrics.extract_entity(pred)

        if None in (gold_entity, pred_entity):
            raise ValueError(f"Gold ({gold_entity}) or prediction ({pred_entity}) has wrong format or None")
        
        return 1. if gold_entity == pred_entity else 0.
    
    @staticmethod
    def partial_match_levenshtein(gold: str, pred: str) -> float:
        """
        Частичное совпадение границ и формы сущности (расстояние Левенштейна),
        без учёта типа
    
        Args:
            gold (str | None): истинное значение, строка вида "entity|type"
            pred (str | None): предсказанное значение, строка вида "entity|type"
        Returns:
            score (float): нормализованное сходство от 0.0 до 1.0, где:
                1.0 - полное сходство сущностей
                0.0 - полное несовпадение
        """
        if pd.isna(gold) and pd.isna(pred):
            return 1.
        if pd.isna(gold) or pd.isna(pred):
            return 0.
        
        # Извлечение сущностей
        gold_entity: str = NERMetrics.extract_entity(gold)  # type: ignore
        pred_entity: str = NERMetrics.extract_entity(pred)  # type: ignore

        if None in (gold_entity, pred_entity):
            raise ValueError(f"Gold ({gold_entity}) or prediction ({pred_entity}) has wrong format or None")
        
        if gold_entity == pred_entity:
            return 1.
        
        # Вычисляем расстояние Левенштейна
        distance = NERMetrics.levenshtein_distance(gold_entity, pred_entity)
        max_len = max(len(gold_entity), len(pred_entity))

        # Нормализованное расстояние
        return max(0., 1. - distance / max_len)

    @staticmethod
    def levenshtein_distance(s1: str, s2: str) -> float:
        """
        Вычисляет модифицированное расстояние Левенштейна между двумя строками
        (несовпадение регистра даёт штраф 0.5)
        
        Args:
            s1 (str): первая строка
            s2 (str): вторая строка
        Returns:
            distance (float): минимальное количество операций (вставка, удаление, замена)
                для преобразования s1 в s2. Смена регистра стоит 0.5 вместо 1.0
        """

        if len(s1) < len(s2):
            return NERMetrics.levenshtein_distance(s2, s1)
        
        if len(s2) == 0:
            return len(s1)
        
        # Динамическое программирование
        previous_row = list(range(len(s2) + 1))
        for i, c1 in enumerate(s1):
            current_row = [i + 1.]
            for j, c2 in enumerate(s2):
                # Стоимость операций
                insertions = previous_row[j + 1] + 1.      # вставка
                deletions = current_row[j] + 1.            # удаление

                if c1 == c2:
                    cost = 0.
                elif c1.lower() == c2.lower():  # Разный регистр
                    cost = 0.5
                else:
                    cost = 1.0

                substitutions = previous_row[j] + cost  # замена
                current_row.append(min(insertions, deletions, substitutions))
            previous_row = current_row
        
        return previous_row[-1]
    
    @staticmethod
    def type_accuracy_score(gold: str, pred: str) -> float:
        """
        Точность типа (совпадение или нет)

        Args:
            gold (str | None): истинное значение, строка вида "entity|type"
            pred (str | None): предсказанное значение, строка вида "entity|type"
        Returns:
            score (float): 1.0 если типы совпадают, 
                          0.0 если типы не совпадают
        """

        if pd.isna(gold) and pd.isna(pred):
            return 1.
        if pd.isna(gold) or pd.isna(pred):
            return 0.
        
        gold_type = NERMetrics.extract_type(gold)
        pred_type = NERMetrics.extract_type(pred)

        if None in (gold_type, pred_type):
            raise ValueError(f"Gold ({gold_type}) or prediction ({pred_type}) has wrong format or None")

        return 1. if gold_type == pred_type else 0.
    
    @staticmethod
    def compute_precision_recall_f1(tp: int, fp: int, fn: int) -> Tuple[float, float, float]:
        """
        Вычисляет precision, recall и F1-score

        Args:
            tp: True Positives count
            fp: False Positives count
            fn: False Negatives count
        Returns:
            metrics : Tuple[precision, recall, f1]
        """
        if tp + fp == 0:
            return 0.0, 0.0, 0.0
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2 * precision * recall / (precision + recall)

        return precision, recall, f1

    @staticmethod
    def score_fn(gold: str, pred: str, type_importance: float=1.0) -> float:
        """
        Основная функция подсчета метрики для NER
        Вычисляет сходство найденной сущности от 0.0 до 1.0 на основе расстояния Левенштейна
        Если же тип выбран неверно, то метрика домножается на 
        
        Args:
            gold: золотой стандарт в формате "entity|type" или None
            pred: предсказание в формате "entity|type" или None
            type_importance (от 0.0 до 1.0): коэффициент важности типа для метрики 
                (умножается на близость по Левенштейну)
            
        Returns:
            score (float): оценка от 0.0 до 1.0, где 1.0 - точное совпадение
        """
        wrong_type_penalty = type_importance * (1 - NERMetrics.type_accuracy_score(gold, pred))
        return NERMetrics.boundary_match_score(gold, pred) * wrong_type_penalty

    @staticmethod
    def vectorized_ner_evaluation(df: pd.DataFrame) -> dict:
        """
        Векторизованное вычисление метрик NER для DataFrame

        Args:
            df: DataFrame с колонками 'entity', 'gold_answer', 'llm_entity', 'llm_answer'

        Returns:
            metrics_info (dict): словарь с полученными метриками
        """
        # Подготовка данных - объединение entity и type
        def combine_entity_type(entity, entity_type):
            if pd.isna(entity) or pd.isna(entity_type):
                return None
            return f"{entity}|{entity_type}"

        # Векторизованное объединение
        gold_combined = df.apply(lambda row: combine_entity_type(row['entity'], row['gold_answer']), axis=1)  # type: ignore
        pred_combined = df.apply(lambda row: combine_entity_type(row['llm_entity'], row['llm_answer']), axis=1)  # type: ignore
        
        # Векторизованное вычисление метрик
        exact_scores = np.vectorize(NERMetrics.exact_match_score)(gold_combined, pred_combined)
        boundary_scores = np.vectorize(NERMetrics.boundary_match_score)(gold_combined, pred_combined)
        partial_scores = np.vectorize(NERMetrics.partial_match_levenshtein)(gold_combined, pred_combined)
        type_scores = np.vectorize(NERMetrics.type_accuracy_score)(gold_combined, pred_combined)

        # Вектора булевых занчений наличия сущности
        gold_has_entity = ~pd.isna(gold_combined)
        pred_has_entity = ~pd.isna(pred_combined)

        """ Подсчёт TP, FP, FN для разных метрик """

        # 1. Точное совпадение сущности и типа
        exact_tp = np.sum(exact_scores == 1. & pred_has_entity & gold_has_entity)
        exact_fp = np.sum(pred_has_entity & (exact_scores == 0.))
        exact_fn = np.sum(gold_has_entity & (exact_scores == 0.))

        # 2. Совпадение границ и форм сущностей (только entity)
        boundary_tp = np.sum(boundary_scores == 1. & pred_has_entity & gold_has_entity)
        boundary_fp = np.sum(pred_has_entity & (boundary_scores == 0.))
        boundary_fn = np.sum(gold_has_entity & (boundary_scores == 0.))

        # 3. Частичное совпадение (средний score)
        partial_avg = np.mean(partial_scores)
        
        # 4. Точность типов (только при точном совпадении сущностей)        
        boundary_match_mask = boundary_scores == 1.0  # Фильтруем только случаи, где границы сущностей совпадают
        valid_type_scores = type_scores[boundary_match_mask]
        type_accuracy = np.mean(valid_type_scores) if len(valid_type_scores) > 0 else 0.0

        # Вычисляем precision, recall, F1
        exact_precision, exact_recall, exact_f1 = \
            NERMetrics.compute_precision_recall_f1(exact_tp, exact_fp, exact_fn)
        
        boundary_precision, boundary_recall, boundary_f1 = \
            NERMetrics.compute_precision_recall_f1(boundary_tp, boundary_fp, boundary_fn)
        
        # Метрики по классам
        class_metrics = {}
        for entity_type in ['PER', 'ORG', 'LOC', 'EVT', 'PRO']:
            # Фильтруем только данный класс
            gold_class_mask = df['gold_answer'] == entity_type
            pred_class_mask = df['llm_answer'] == entity_type
            
            class_tp = np.sum(gold_class_mask & pred_class_mask & (exact_scores == 1.0))
            class_fp = np.sum(pred_class_mask & (exact_scores == 0.0))
            class_fn = np.sum(gold_class_mask & (exact_scores == 0.0))
            
            class_precision, class_recall, class_f1 = \
                NERMetrics.compute_precision_recall_f1(class_tp, class_fp, class_fn)
            
            class_metrics[entity_type] = {
                'precision': class_precision,
                'recall': class_recall,
                'f1': class_f1,
                'gold_count': np.sum(gold_class_mask)
            }
        
        # Macro F1
        macro_f1 = np.mean([metrics['f1'] for metrics in class_metrics.values()])
        
        return {
            # Общие метрики
            'exact_match_precision': exact_precision,
            'exact_match_recall': exact_recall,
            'exact_match_f1': exact_f1,
            'boundary_precision': boundary_precision,
            'boundary_recall': boundary_recall,
            'boundary_f1': boundary_f1,
            'partial_match_avg_similarity': partial_avg,
            'type_accuracy': type_accuracy,
            'macro_f1': macro_f1,
            
            # Статистика
            'total_entities_gold': np.sum(gold_has_entity),
            'total_entities_pred': np.sum(pred_has_entity),
            'exact_tp': int(exact_tp),
            'exact_fp': int(exact_fp),
            'exact_fn': int(exact_fn),
            
            # По классам
            'class_metrics': class_metrics
        }